In [1]:
import os
import utils

utils.load_env()
os.environ['LANGCHAIN_TRACING_V2'] = "true"

In [2]:
from typing import Annotated, Literal, TypedDict

from langchain_core.messages import HumanMessage

# for llm model
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
import tools


tool_node = tools.tool_node

# model = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0).bind_tools(tools)
model = ChatOpenAI(model="gpt-4o-mini")

# Define the function that determines whether to continue or not
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END


# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", 'agent')

# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable.
# Note that we're (optionally) passing the memory when compiling the graph
app = workflow.compile(checkpointer=checkpointer)

In [5]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="ค้นหาร้านกาแฟใกล้มาบุญครอง")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')


'บริเวณมาบุญครองในกรุงเทพฯ มีร้านกาแฟหลายแห่งที่น่าสนใจ นี่คือบางร้านที่คุณอาจจะสนใจ:\n\n1. **ร้านกาแฟดอยช้าง** - เป็นร้านกาแฟที่มีชื่อเสียงในเรื่องของกาแฟคุณภาพสูงจากดอยช้าง\n2. **ร้านกาแฟอเมซอน** - ร้านกาแฟยอดนิยมที่มีสาขาหลายแห่งทั่วประเทศ\n3. **ร้านกาแฟ Starbucks** - สาขา Starbucks มักจะมีอยู่ในห้างสรรพสินค้าหรือบริเวณใกล้เคียง\n4. **ร้านกาแฟท้องถิ่น** - มักจะมีร้านเล็ก ๆ ที่เสิร์ฟกาแฟสดใหม่และมีบรรยากาศน่ารัก\n\nคุณสามารถใช้แอปพลิเคชันหรือเว็บไซต์ค้นหาร้านกาแฟ เช่น Google Maps เพื่อค้นหาร้านกาแฟที่ใกล้ที่สุดและดูรีวิวเพิ่มเติมได้ค่ะ'